###Imports

In [0]:
from google.colab import drive
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
import csv
import glob
import os
import pandas as pd
import time

### **Preprocessing**
##Iterate over frames in each video to:
1. Crop the face using haarcascade face detector.
2. Calculate flow between each two consecutive frames
3. Save the cropped image and the flow image.

In [0]:
from google.colab import drive
drive.mount('/content/drive')
home_dir = '/content/drive/My Drive/Colab Notebooks/FER/'
baum_dir = home_dir + 'BAUM/'
temp_image = np.array([])
none_image = None

videos = glob.glob(baum_dir + 'BAUM1s_MP4 - All/*/*.mp4')
videos = sorted(videos)
print(len(videos), " videos was detected in baum_dir")


# Create a face detec tion module 
calssifiers =['haarcascade_frontalface_default.xml',
              'haarcascade_frontalface_alt2.xml',
              'haarcascade_frontalcatface_extended.xml',
              'haarcascade_frontalcatface.xml',
              'haarcascade_frontalface_alt_tree.xml',
              'haarcascade_frontalface_alt.xml']

def detectFace(img):
    for classifier in calssifiers:
        face_cascade = cv2.CascadeClassifier(home_dir +'face_detectors/' + classifier)
        faces = face_cascade.detectMultiScale(img, 1.3, 1)
        if (len(faces) > 0):
            return faces
    faces = []
    return faces

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
1198  videos was detected in baum_dir


In [0]:
stamp = str(time.time()).split('.')[0]
csv_file_name = 'data_' + stamp +".csv"
#csv_file_name = baum_dir + csv_file_name
#print ("Target scv file is: ", csv_file_name)

data_generated = 0
if not data_generated:
  #data_frame = pd.DataFrame([["flow", "spatial",  "emotion", "code"]])
  #data_frame.to_csv(csv_file_name, mode='w', header=False, index = False, sep = ',')
  data_labels_xl =  pd.read_excel(baum_dir +"Annotations_BAUM1s.xlsx")
  target_codes = [1, 6, 7, 10, 11, 8]
  data_labels = np.array(data_labels_xl)

  videoSkipped = 0
  allbadFrames_count = 0
  videoProcessed = 0
  for video in videos:
      # Test if this video has been already procedded
      already_processed = 0
      flow_path = video.replace('BAUM1s_MP4 - All', 'imgs_flow').replace('.mp4', '/')
      spatial_path = video.replace('BAUM1s_MP4 - All', 'imgs_spatial').replace('.mp4', '/')
      flow_relative_path = flow_path.split('/')[-4] + '/' + flow_path.split('/')[-3] + '/' + flow_path.split('/')[-2] 
      spatial_relative_path = spatial_path.split('/')[-4] + '/' + spatial_path.split('/')[-3] + '/' + spatial_path.split('/')[-2]
      folder = baum_dir + spatial_relative_path
      contents = glob.glob(folder + "/" + "*.png")
      data_count = len(contents)
      if  data_count > 15:
          already_processed = 1

      # Test if this video in one of the 6 classes
      sample_name = video.split('/')[-1].split('.')[0]
      label_found = 0
      for label in data_labels:
        if label[3] == sample_name:
            emotion = label[4]
            code = label[5]
            if code in target_codes:
              label_found = 1
              break

      if (not label_found) or (already_processed):
        pass #print("already processed or out of classes")
      else:
        #print(video)
        videoProcessed += 1
        cap = cv2.VideoCapture(video)
        no_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  
        ret, old_frame = cap.read()
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

        faces = detectFace(old_gray)
        count = 0
        while (len(faces) == 0):    
            if count > no_of_frames - 1: #Stop looking for a valid frame at the end of the video #
                break
            count+=1
            #print("First Frame is not good for face detection ", count, no_of_frames)
            ret, old_frame = cap.read()
            if isinstance(old_frame, type(temp_image)) and not isinstance(old_frame, type(none_image)) :
                old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
                faces = detectFace(old_gray)
            else:
                print ("bad frame", type(old_frame), video)
                
                      
        if (len(faces) == 0):
            print ("All Video Frames are BAD", video)
            videoSkipped += 1
            continue
        
        (x,y,w,h) = faces[0]
        old_gray = old_gray[y:y+h, x:x+w]
        old_color = old_frame[y:y+h, x:x+w]
        size_gray = old_gray.shape
        size_color = old_color.shape

        # Create a mask image for drawing purposes
        mask = np.zeros_like(old_color)
        hsv = np.zeros_like(old_color)
        hsv[...,1] = 255
        
        noFacesCnt = 0
        calcOpticalFlow = 0
        for i in range(no_of_frames - 1):
            ret,frame = cap.read()
            if isinstance(frame, type(temp_image)) and not isinstance(frame, type(none_image)) :
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                faces = detectFace(frame_gray)
                if len(faces) == 0:
                    calcOpticalFlow = 0
                    noFacesCnt += 1
                    allbadFrames_count +=1
                    #print ("No Faces Found ", noFacesCnt, no_of_frames, video)
                    continue
                
                (x,y,w,h)  = faces[0]
                frame_gray = frame_gray[y:y+h, x:x+w]
                roi_color = frame[y:y+h, x:x+w]
                
                if(frame_gray.shape[0] < 90 or roi_color.shape[0] < 90 ):
                    calcOpticalFlow = 0
                    allbadFrames_count +=1
                    #print("Skip this frame , small image!! ", video , i)
                    continue
                                
                #print(frame_gray.shape)
                frame_gray = cv2.resize(frame_gray, (size_gray[1],size_gray[0]))
                roi_color = cv2.resize(roi_color, (size_color[1],size_color[0]))
                #print(frame_gray.shape, "after")
              
                calcOpticalFlow += 1
                            
                if (calcOpticalFlow > 1):
                    flow = cv2.calcOpticalFlowFarneback(old_gray,frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)            
                    mag, ang   = cv2.cartToPolar(flow[...,0], flow[...,1])
                    hsv[...,0] = ang*180/np.pi/2
                    hsv[...,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
                    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
                    
                    str_i = '{:03}'.format(i) # max 999 frames

                    flow_relative_name = flow_relative_path + '/' + str_i + '.png'
                    spatial_relative_name = spatial_relative_path + '/' + str_i + '.png'
                    
                    flow_name_abs = baum_dir + flow_relative_name
                    spatial_name_abs = baum_dir + spatial_relative_name

                    flow_folder = flow_name_abs.replace('/'+str_i+'.png', '')
                    spatial_folder = spatial_name_abs.replace('/'+str_i+'.png', '')

                    if not os.path.isdir(flow_folder):
                        os.makedirs(flow_folder)
                    if not os.path.isdir(spatial_folder):
                        print("create folder: ", spatial_folder)
                        os.makedirs(spatial_folder)
                    
                    cv2.imwrite(flow_name_abs, rgb)
                    cv2.imwrite(spatial_name_abs,roi_color)
                    print("write Image to directory", video)
                    #data_frame =  pd.DataFrame([[flow_relative_name, spatial_relative_name,  emotion, str(code)]])
                    #data_frame.to_csv(csv_file_name, mode='a', header=False, index = False, sep = ',')
                        
                # Now update the previous frame and previous points
                old_gray = frame_gray.copy()
              
  print("No of skipped videos is: ", videoSkipped)
  print("No of allbadFrames is: ", allbadFrames_count)
  print("No of video processed is ", videoProcessed)

# Get All Written Images

In [0]:
from google.colab import drive
drive.mount('/content/drive')

from glob import glob
import os, time
import pandas as pd

home_dir = '/content/drive/My Drive/Colab Notebooks/FER/'
baum_dir = home_dir + 'BAUM/'

data_labels_xl =  pd.read_excel(baum_dir +"Annotations_BAUM1s.xlsx")

stamp = str(time.time()).split('.')[0]
csv_file_name = 'data_6_classes.csv'
csv_file_name = baum_dir + csv_file_name
data_frame = pd.DataFrame([["flow", "spatial",  "emotion", "code"]])
data_frame.to_csv(csv_file_name, mode='w', header=False, index = False, sep = ',')

print ("Target scv file is: ", csv_file_name)

flow = baum_dir + 'imgs_flow/'
spatial = baum_dir + 'imgs_spatial/'

flowImgs = glob(flow+'*/*/*.png')
count = 0
for img in flowImgs:
    flowImg = img
    spatialImg = img.replace('imgs_flow', 'imgs_spatial')
    if (os.path.isfile(spatialImg)):
        count += 1
        vidName = img.split('/')[-2]
        vidLabel = str(data_labels_xl.loc[data_labels_xl['Clip Name'] == vidName]['Emotion'].values[0])
        vidCode = str(data_labels_xl.loc[data_labels_xl['Clip Name'] == vidName]['Emotion Code'].values[0])
        if (int(vidCode) in (1, 10, 11, 6, 7, 8)):
            #print ('Needed Emotion')
            print (flowImg, spatialImg, vidLabel, vidCode)
            data_frame =  pd.DataFrame([[flowImg, spatialImg,  vidLabel, vidCode]])
            data_frame.to_csv(csv_file_name, mode='a', header=False, index = False, sep = ',')

print (len(flowImgs), count)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Target scv file is:  /content/drive/My Drive/Colab Notebooks/FER/BAUM/data_1583416240.csv


### **Splitting data to train / valid**

In [0]:
import pandas as pd
import numpy as np
home_dir = '/content/drive/My Drive/Colab Notebooks/FER/'
baum_dir = home_dir + 'BAUM/'

csv_file = 'data_6_classes.csv'
data = pd.read_csv(baum_dir + csv_file )
data.sort_values(by=['code'])
data = np.array(data)

training_data = baum_dir + "training_data.csv"
validation_data = baum_dir + "validation_data.csv"


data_frame = pd.DataFrame([["flow", "spatial",  "emotion", "code"]])
data_frame.to_csv(training_data, mode='w', header=False, index = False, sep = ',')
data_frame.to_csv(validation_data, mode='w', header=False, index = False, sep = ',')

def write_to_file(seg, valid): 
    data_frame =  pd.DataFrame([[seg[0], seg[1],  seg[2], seg[3]]])  
    if valid:
        data_frame.to_csv(validation_data, mode='a', header=False, index = False, sep = ',')
    else: 
        data_frame.to_csv(training_data, mode='a', header=False, index = False, sep = ',')
  
val = 8 # 7 train to 1 valid
last_folder = ""   
for row in data:
    file_name = row[0]
    #print(file_name)
    folder = file_name.split('/')[-3]
    frame = [row[0], row[1], row[2], row[3]]
 
    if ( folder != last_folder ):
        val -= 1
        if val == 0:
            valid = 1
            write_to_file(frame, 1)
            print("valid")
            val = 8
        else:
            valid = 0
            print("train")
            write_to_file(frame, 0)
    elif valid:
        write_to_file(frame, 1)
    elif not valid:
        write_to_file(frame, 0)
    last_folder = folder
    
